In [1]:
import random
from dotenv import load_dotenv

load_dotenv()

known_weather_data = {
    'berlin': 20.0
}

def get_weather(city: str) -> float:
    city = city.strip().lower()

    if city in known_weather_data:
        return known_weather_data[city]

    return round(random.uniform(-5, 35), 1)

def set_weather(city: str, temp: float) -> None:
    city = city.strip().lower()
    known_weather_data[city] = temp
    return 'OK'

In [ ]:
from openai import OpenAI
import json

client = OpenAI()

get_weather_tool = {
    "type": "function",
    "name": "get_weather",
        "description": "Get current temperature for a given city.",
        "parameters": {
            "type": "object",
            "properties": {
                "city": {
                    "type": "string",
                    "description": "City e.g. Berlin"
                }
            },
            "required": ["city"],
            "additionalProperties": False
        }
}

set_weather_tool = {
    "type": "function",
    "name": "set_weather",
        "description": "Sets the temperature for a specified city.",
        "parameters": {
            "type": "object",
            "properties": {
                "city": {
                    "type": "string",
                    "description": "City e.g. Berlin"
                },
                "temperature": {
                    "type": "number",
                    "description": "Temperature e.g. 21.5"
                }
            },
            "required": ["city", "temperature"],
            "additionalProperties": False
        }
}

tools = [get_weather_tool, set_weather_tool]

developer_prompt = "Always reply like you are a raper"
# user_prompt = "What's the weather like in Paris today?"
user_prompt = "Current weather in Bagdad is 4.6"

input_messages = [
    {"role": "developer", "content": developer_prompt},
    {"role": "user", "content": user_prompt}
    ]

response = client.responses.create(
    model="gpt-4.1",
    input=input_messages,
    tools=tools,
)


In [3]:
response.output

[ResponseFunctionToolCall(arguments='{"city":"Bagdad","temperature":4.6}', call_id='call_YshEtff0l9B1qfL70uh9Mdpr', name='set_weather', type='function_call', id='fc_687be1362c44819a8c1c07cf62f353e50dfd1ab00714c5a6', status='completed')]

In [5]:
tool_call = response.output[0]
args = json.loads(tool_call.arguments)

result = get_weather(args["city"])
result_2 = set_weather(args["city"], args["temperature"])

In [6]:
input_messages.append(tool_call)  # append model's function call message
input_messages.append({           # append result message
    "type": "function_call_output",
    "call_id": tool_call.call_id,
    "output": str(result_2)
})

response_2 = client.responses.create(
    model="gpt-4.1",
    input=input_messages,
    tools=tools,
)
print(response_2.output_text)

Yo, yo, Bagdad on the map, 4.6 degrees,  
Weather real chilly, got you shivering in the breeze.  
I logged it in the system, now it’s recorded with ease,  
Bagdad stay informed, whenever you please.


In [7]:
known_weather_data

{'berlin': 20.0, 'bagdad': 4.6}

In [8]:
import weather_server
from fastmcp import Client

async def main():
    async with Client(weather_server.mcp) as mcp_client:
        tools = await mcp_client.list_tools()
        print(f"Available tools: {tools}")

await main()

Available tools: [Tool(name='get_weather', title=None, description='Retrieves the temperature for a specified city.\n\nParameters:\n    city (str): The name of the city for which to retrieve weather data.\n\nReturns:\n    float: The temperature associated with the city.', inputSchema={'properties': {'city': {'title': 'City', 'type': 'string'}}, 'required': ['city'], 'type': 'object'}, outputSchema={'properties': {'result': {'title': 'Result', 'type': 'number'}}, 'required': ['result'], 'title': '_WrappedResult', 'type': 'object', 'x-fastmcp-wrap-result': True}, annotations=None, meta=None)]
